# Ticket Issue Recommendation

### clustering the data into 100 places

#### Read Data

In [62]:
import pandas as pd
import numpy as np
df = pd.read_csv("NYPD_Motor_Vehicle_Collisions.csv")

In [63]:
# Data column
df.columns

Index([u'DATE', u'TIME', u'BOROUGH', u'ZIP CODE', u'LATITUDE', u'LONGITUDE',
       u'LOCATION', u'ON STREET NAME', u'CROSS STREET NAME',
       u'OFF STREET NAME', u'NUMBER OF PERSONS INJURED',
       u'NUMBER OF PERSONS KILLED', u'NUMBER OF PEDESTRIANS INJURED',
       u'NUMBER OF PEDESTRIANS KILLED', u'NUMBER OF CYCLIST INJURED',
       u'NUMBER OF CYCLIST KILLED', u'NUMBER OF MOTORIST INJURED',
       u'NUMBER OF MOTORIST KILLED', u'CONTRIBUTING FACTOR VEHICLE 1',
       u'CONTRIBUTING FACTOR VEHICLE 2', u'CONTRIBUTING FACTOR VEHICLE 3',
       u'CONTRIBUTING FACTOR VEHICLE 4', u'CONTRIBUTING FACTOR VEHICLE 5',
       u'UNIQUE KEY', u'VEHICLE TYPE CODE 1', u'VEHICLE TYPE CODE 2',
       u'VEHICLE TYPE CODE 3', u'VEHICLE TYPE CODE 4', u'VEHICLE TYPE CODE 5'],
      dtype='object')

#### Process the data into Month/Day/Year

In [64]:
# Create dataset
processedDf = pd.DataFrame()
processedDf['Date'] = df['DATE']
processedDf['Time'] = df['TIME']
processedDf['latitude'] = df['LATITUDE']
processedDf['longitude'] = df['LONGITUDE'] #Suppose to add a column of reason here

In [65]:
clusterdata = processedDf[processedDf['latitude'].notnull()]

In [66]:
# append year month day
getyear = []
getmonth = []
getday = []

def datesplit(raw):
    getyear.append(raw.split("/")[2])
    getday.append(raw.split("/")[1])
    getmonth.append(raw.split("/")[0])

clusterdata['Date'].apply(datesplit)

clusterdata = clusterdata.assign(month = getmonth, day = getday, year = getyear)

In [67]:
# append week
import datetime

getweek = []
def computeweek(year, month, day):
    getweek.append(datetime.date(int(year), int(month), int(day)).weekday())
    
for row in clusterdata.iterrows():
    computeweek(row[1]['year'], row[1]['month'], row[1]['day'])

clusterdata = clusterdata.assign(week = getweek) # 0 = Monday

In [9]:
len(clusterdata.index)

932164

In [68]:
# convert time to minutes based
time_in_min = []

def convertime(raw):
    hs = raw.split(":")
    time_in_min.append(int(hs[0])*60 + int(hs[1]))

clusterdata['Time'].apply(convertime)

clusterdata = clusterdata.assign(tominutes = time_in_min)

In [69]:
clusterdata = clusterdata.drop(['Time', 'Date', 'day', 'year'], axis = 1)

##### Normalize the data, since we want to put more value on the position and careless about the month/week/time

In [ ]:
clusterdata = clusterdata.assign(tominutes = time_in_min) # Not finished, do the normalization here

#### Do the cluster

##### Use the KMeans to do the clustering

In [72]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=100, random_state=0).fit(clusterdata)

In [13]:
import pickle

def saveObject(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
saveObject(kmeans, "kmeasModel")

In [53]:
kmeans.cluster_centers_.astype('float16')
array = ['40.7555', '-73.0384', '6', '3', '770']
kmeans.predict([array])

array([18], dtype=int32)

In [54]:
labels = kmeans.labels_
class_member_mask = (labels == 2)
print class_member_mask, clusterdata[class_member_mask]

[False False False ..., False False False]           latitude  longitude month  week  tominutes
346      40.712616 -73.969510    10     5         70
347      40.660490 -73.908390    10     5         75
348      40.696090 -73.896320    10     5         75
349      40.712864 -73.797140    10     5         75
1611     40.718792 -73.989020    10     3         65
2832     40.870106 -73.878720    10     1         75
3492     40.847897 -73.924990    10     0         75
4602     40.723940 -73.976620    10     5         72
5321     40.625973 -73.976240    10     4         75
5739     40.729880 -73.749520    06     4         75
6073     40.777096 -73.767680    10     3         75
6811     40.681286 -73.769790    10     2         75
7541     40.595360 -73.800940    10     1         70
8182     40.760240 -73.803870    10     0         70
8707     40.777874 -73.951750    10     6         70
8708     40.824432 -73.873604    10     6         70
8709     40.761720 -73.982810    10     6         75
871

In [61]:
kmeans.cluster_centers_[18]

array([  40.72136923,  -73.94077045,    6.49073369,    2.84683084,
        765.0383346 ])

##### Use the DBSCAN to do the clustering

In [36]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps = 1, min_samples = 100).fit(clusterdata)

In [59]:
db.core_sample_indices_

array([    75,     76,     77, ..., 932008, 932009, 932010])

In [39]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print n_clusters_, set(labels)

326 set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 2

In [43]:
unique_labels = set(labels)
for k in unique_labels:
    if k == -1:
        print "k = -1"
    class_member_mask = (labels == k)
    print clusterdata[class_member_mask & core_samples_mask]


          latitude  longitude month  week  tominutes
81       40.858196 -73.870990    10     5        720
82       40.607450 -73.950005    10     5        720
83       40.745285 -73.948310    10     5        720
84       40.764015 -73.871660    10     5        720
85       40.639650 -74.109880    10     5        720
86       40.707954 -73.837150    10     5        720
87       40.714440 -73.831270    10     5        720
88       40.766743 -73.925970    10     5        720
4325     40.861390 -73.870250    10     5        720
4326     40.889880 -73.902940    10     5        720
4327     40.749300 -73.897580    10     5        720
4328     40.701244 -73.991530    10     5        720
4329     40.757973 -73.834220    10     5        720
4330     40.785690 -73.787200    10     5        720
8968     40.728424 -73.953430    10     5        720
8969     40.711530 -73.852930    10     5        720
8970     40.703934 -73.959854    10     5        720
13192    40.744762 -73.995170    10     5     

          latitude  longitude month  week  tominutes
246      40.831924 -73.889340    10     5       1020
247      40.610313 -74.003075    10     5       1020
248      40.727710 -73.978070    10     5       1020
249      40.742380 -74.008820    10     5       1020
251      40.667976 -73.798140    10     5       1020
252      40.638313 -74.006630    10     5       1020
253      40.834420 -73.862970    10     5       1020
254      40.846725 -73.935630    10     5       1020
255      40.634743 -73.964940    10     5       1020
835      40.855370 -73.904730    10     4       1020
836      40.809628 -73.890045    10     4       1020
837      40.628647 -74.025960    10     4       1020
839      40.667885 -73.974130    10     4       1020
840      40.763817 -73.880104    10     4       1020
841      40.671658 -73.742130    10     4       1020
843      40.667480 -73.921880    10     4       1020
844      40.746650 -73.969124    10     4       1020
845      40.750930 -73.897510    10     4     

          latitude  longitude month  week  tominutes
626      40.856020 -73.888460    10     4        750
628      40.725080 -73.820990    10     4        750
629      40.656210 -73.914665    10     4        750
630      40.630173 -74.000755    10     4        750
631      40.682160 -73.960600    10     4        750
632      40.821750 -73.913070    10     4        750
4925     40.883050 -73.899830    10     4        750
9531     40.680733 -73.974510    10     4        750
9532     40.706640 -74.012640    10     4        750
9533     40.784610 -73.969840    10     4        750
9534     40.545723 -74.230125    10     4        750
13763    40.663440 -73.950920    10     4        750
13764    40.748234 -73.941670    10     4        750
13765    40.722010 -73.869070    10     4        750
13766    40.833836 -73.915860    10     4        750
13767    40.749416 -73.884250    10     4        750
13768    40.844746 -73.903630    10     4        750
13769    40.844917 -73.926384    10     4     

          latitude  longitude month  week  tominutes
1143     40.734375 -73.873420    10     4        490
1758     40.626877 -74.173740    10     3        490
2355     40.767060 -73.739120    10     2        490
2356     40.840824 -73.945970    10     2        490
2997     40.737500 -73.852760    10     1        490
5506     40.818764 -73.959800    10     4        490
6251     40.609680 -73.945360    10     3        490
6252     40.613056 -73.917570    10     3        490
6253     40.754105 -73.723210    10     3        490
6254     40.876730 -73.847305    10     3        490
6981     40.859410 -73.892265    10     2        490
6982     40.733400 -74.008560    10     2        490
6983     40.771390 -73.950440    10     2        490
6984     40.862747 -73.905120    10     2        490
6985     40.588005 -73.983630    10     2        490
10617    40.714005 -73.941720    10     3        490
10618    40.634130 -73.928505    10     3        490
11234    40.804012 -73.930970    10     2     

          latitude  longitude month  week  tominutes
1861     40.843002 -73.903590    10     2        670
1862     40.822258 -73.887020    10     2        670
6408     40.737797 -73.850700    10     2        670
7136     40.857950 -73.927270    10     1        670
7847     40.834114 -73.863270    10     0        670
10150    40.647182 -73.907480    10     3        670
10151    40.810406 -73.906700    10     3        670
10727    40.615800 -73.930680    10     2        670
10728    40.577766 -73.956060    10     2        670
11373    40.754500 -73.971950    10     1        670
11374    40.611774 -74.070690    10     1        670
12110    40.796800 -73.929375    10     0        670
14369    40.757286 -73.978120    10     3        670
14370    40.757725 -73.863720    10     3        670
16410    40.766490 -73.956960    10     0        670
225983   40.814937 -73.914954    10     0        670
225984   40.684381 -73.975046    10     0        670
225985   40.652314 -73.956011    10     0     

...

          latitude  longitude month  week  tominutes
188916   40.743448 -74.003540    12     3        540
188917   40.658974 -73.727253    12     3        540
188920   40.717014 -73.829710    12     3        540
189463   40.878045 -73.825862    12     2        540
189464   40.729907 -73.979077    12     2        540
189465   40.690731 -73.775490    12     2        540
192725   40.661976 -73.859987    12     3        540
192726   40.686817 -73.951813    12     3        540
192727   40.689710 -73.804207    12     3        540
192734   40.715849 -73.809901    12     3        540
192735   40.743141 -73.749799    12     3        540
192737   40.649530 -73.916330    12     3        540
193386   40.689432 -73.996874    12     2        540
193387   40.709325 -73.937611    12     2        540
193388   40.634660 -74.034891    12     2        540
193389   40.689647 -73.918399    12     2        540
193390   40.627857 -73.904093    12     2        540
193392   40.740879 -73.990069    12     2     

          latitude  longitude month  week  tominutes
190022   40.716871 -73.995686    12     1        510
194085   40.617992 -73.982134    12     1        510
194087   40.740853 -73.883299    12     1        510
198800   40.710957 -73.951117    12     1        510
198801   40.621509 -73.913176    12     1        510
198802   40.706309 -74.003040    12     1        510
198803   40.755714 -73.878776    12     1        510
198804   40.733847 -73.871578    12     1        510
198805   40.721064 -73.759442    12     1        510
416688   40.852488 -73.927695    12     1        510
416689   40.778883 -73.988312    12     1        510
420496   40.827758 -73.911693    12     1        510
420497   40.677313 -73.899126    12     1        510
420498   40.635447 -73.998851    12     1        510
420499   40.639735 -73.906742    12     1        510
420500   40.770480 -73.983855    12     1        510
420501   40.685018 -73.843353    12     1        510
420502   40.629344 -74.127195    12     1     

          latitude  longitude month  week  tominutes
206762   40.839240 -73.946863    11     2        911
206786   40.820189 -73.890675    11     2        911
206787   40.893053 -73.899898    11     2        911
206788   40.621994 -73.945970    11     2        911
206789   40.666277 -73.934140    11     2        911
206790   40.659104 -73.962577    11     2        911
206791   40.580629 -73.966210    11     2        911
206792   40.700009 -73.960239    11     2        911
206793   40.745727 -73.978123    11     2        911
206794   40.739173 -73.976862    11     2        911
206795   40.755075 -73.988684    11     2        911
206796   40.813012 -73.945392    11     2        911
206797   40.792241 -73.936413    11     2        911
206798   40.753652 -73.978672    11     2        911
206799   40.719137 -73.745390    11     2        911
206813   40.621493 -74.170754    11     2        911
206814   40.818046 -73.922457    11     2        911
206815   40.654791 -73.925851    11     2     

         latitude  longitude month  week  tominutes
208095  40.828099 -73.920202    11     0        611
208096  40.824404 -73.892403    11     0        611
208097  40.654316 -73.922158    11     0        611
208099  40.612845 -73.982503    11     0        611
208100  40.622827 -73.938412    11     0        611
208101  40.701400 -73.958070    11     0        611
208102  40.670887 -73.936489    11     0        611
208103  40.644325 -74.018061    11     0        611
208104  40.735692 -73.956956    11     0        611
208105  40.593323 -73.964638    11     0        611
208106  40.579926 -73.975081    11     0        611
208107  40.679720 -74.015031    11     0        611
208108  40.717280 -73.995429    11     0        611
208109  40.782483 -73.984780    11     0        611
208111  40.809525 -73.937881    11     0        611
208112  40.766104 -73.827690    11     0        611
208113  40.705907 -73.910469    11     0        611
208114  40.705200 -73.869324    11     0        611
208115  40.7

         latitude  longitude month  week  tominutes
685524  40.684278 -73.783015    09     1       1200
[False False False ..., False False False]
          latitude  longitude month  week  tominutes
796057   40.733497 -73.870369    03     4       1110
1008303  40.678156 -73.944151    03     4       1110
[False False False ..., False False False]
          latitude  longitude month  week  tominutes
900336   40.718776 -73.995700    09     6        870
1108112  40.813390 -73.956259    09     6        870
[False False False ..., False False False]
          latitude  longitude month  week  tominutes
917506   40.723308 -74.002980    08     3       1200
1113866  40.741350 -73.981311    08     3       1200
[False False False ..., False False False]
k = -1
Empty DataFrame
Columns: [latitude, longitude, month, week, tominutes]
Index: []
[False False False ..., False False False]


In [29]:
db.p(40.7555, -73.0384, 6, 3, 770)

None


#### Output the model

In [ ]:
import pickle

def saveObject(obj, name):
	with open(name + '.pkl', 'wb') as f:
		pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
	with open(name + '.pkl', 'rb') as f:
		return pickle.load(f)
        